In [1]:
#Importando os módulos H2O e GBM
import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [2]:
#Inicializando o H2O
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/naelia/anaconda3/envs/pibic/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpm6yti702
  JVM stdout: /tmp/tmpm6yti702/h2o_naelia_started_from_python.out
  JVM stderr: /tmp/tmpm6yti702/h2o_naelia_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Fortaleza
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.2
H2O cluster version age:,"28 days, 2 hours and 19 minutes"
H2O cluster name:,H2O_from_python_naelia_gel2cg
H2O cluster total nodes:,1
H2O cluster free memory:,942 Mb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [3]:
#Importando o conjunto de dados do Airlines.
airlines = h2o.import_file('https://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#Convertendo colunas em fatores.
airlines['Year'] = airlines['Year'].asfactor()
airlines['Month'] = airlines['Month'].asfactor()
airlines['DayOfWeek'] = airlines['DayOfWeek'].asfactor()
airlines['Cancelled'] = airlines['Cancelled'].asfactor()
airlines['FlightNum'] = airlines['FlightNum'].asfactor()

In [5]:
#Definindo os nomes do preditor e o nome da coluna de resposta.
predictors = ['Origin', 'Dest', 'Year', 'UniqueCarrier', 'DayOfWeek', 'Month', 'Distance', 'FlightNum']
response = 'IsDepDelayed'

In [6]:
#Dividindo o conjunto de dados em conjuntos de treinamento e validação.
train, valid = airlines.split_frame(ratios = [.8], seed = 1234 )

In [7]:
#Especificando o número de posições que serão incluídas no historograma e depois divididas.
bin_num = [8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]
label = ['8', '16', '32', '64', '128', '256', '512', '1024', '2048', '4096']

In [8]:
#Treinando os modelos.
for key, num in enumerate(bin_num):
    airlines_gbm = H2OGradientBoostingEstimator(nbins_cats = num, seed = 1234)
    airlines_gbm.train(x = predictors, y = response, training_frame = train, validation_frame = valid)


gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%
gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:
#Imprimindo as pontuações da AUC para os dados de treinamento e os dados de validação.
print(label[key], 'training score', airlines_gbm.auc(train = True))
print(label[key], 'validation score', airlines_gbm.auc(valid = True))

4096 training score 0.8592861401336108
4096 validation score 0.7309097636276833


###### Fonte: Copyright 2016-2020 H2O.ai.